In [149]:
%load_ext autoreload
%autoreload 2

import engine

reset graph!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from torchvision import datasets, transforms
train_dataset = datasets.MNIST(
    root='./data', train=True, download=True
)
dataset = train_dataset.data.cpu().numpy()
labels = train_dataset.train_labels.cpu().numpy()

In [81]:
import mediapy
mediapy.show_image(dataset[0], height=512, width=512)

""


In [215]:
import numpy as np
from engine import Variable

scale = .1
weights = [
    Variable("w1", np.random.randn(*(28*28, 28*28+1)) * scale),
    Variable("w2", np.random.randn(*(28*28, 28*28)) * scale),
    Variable("w3", np.random.randn(*(10, 28*28)) * scale),
]

In [ ]:
import engine
engine._reset_graph()
import random

lr = 1e-2

while True:
    # run forward
    idxs = np.random.randint(0, 60_000, size=(1,))
    for idx in idxs:
        x = dataset[idx].reshape((28*28))/255.
        y = labels[idx]

        x_biased = np.concatenate([x, np.ones((1,))])[:, None]
        input = Variable('input', x_biased)

        h = input
        # print(h.value.shape)
        for layer in weights:
            h = engine.Matmul.forward([layer, h])
            h = engine.ReLU.forward([h])
            # print(h.value.shape)
        
        h = engine.Reshape.forward([h], shape=(-1,))
        probs = engine.Softmax.forward([h])
        probs = engine.Reshape.forward([probs], shape=(1,10))
        logprobs = engine.Log.forward([probs])
        
        one_hot = np.zeros((10,1))
        one_hot[y] = -1
        one_hot = Variable('one_hot', one_hot)

        loss = engine.Matmul.forward([logprobs, one_hot])
        print(loss.value)
        print(np.argmax(probs.value).item() == y.item())

        engine.backward(loss)

        for weight in weights:
            grad = engine.GRAPH[weight.name]['grad']
            assert grad.shape == weight.value.shape
            np.subtract(weight.value, lr * grad, out=weight.value)
            
        engine._reset_graph()

In [218]:
logprobs, probs

(Variable(name='5180541113180770272', value=array([[-1.34010003e+01, -1.34010003e+01, -1.36227566e-05,
         -1.34010003e+01, -1.34010003e+01, -1.34010003e+01,
         -1.34010003e+01, -1.34010003e+01, -1.34010003e+01,
         -1.34010003e+01]])),
 Variable(name='-1254749474005433727', value=array([[1.51362931e-06, 1.51362931e-06, 9.99986377e-01, 1.51362931e-06,
         1.51362931e-06, 1.51362931e-06, 1.51362931e-06, 1.51362931e-06,
         1.51362931e-06, 1.51362931e-06]])))

In [219]:
y

np.int64(2)